# Julia on AMD GPU
## AMD製のGPUを用いたインスタンス（amd1dlなど）からJuliaをGPUで駆動させてみましょう！

In [10]:
## N個の一様分布に基づく乱数値(a, b)同士の足し算を行います
## コードはほとんどhttps://juliagpu.gitlab.io/AMDGPU.jl/quickstart/ からのパクリです
## ドキュメントの充実ぶりも好きですJulia
N = 64
a = rand(Float64, N)  # N個の乱数(0 <= n < 1)を生成（配列）
b = rand(Float64, N)
c_cpu = a + b  # c = a + b
println("乱数値同士の和（CPU）: ", c_cpu, "\n")

## 以下では上記と同じ計算をGPUベースで行います。右側のrocm-smiからも目を離さないようにしましょう。
using AMDGPU  # パッケージのimport, 最初は時間かかります

a_d = ROCArray(a)  # GPU演算用の配列に変換
b_d = ROCArray(b)
c_d = similar(a_d)  # a+b演算後の結果を格納するGPU用配列を定義（a_dから）

## 以下の関数では配列の値をスレッド上に配置し、a(a_d)とb(b_d)の値を”一斉に”和演算する
function vadd!(c, a, b)  # 関数の定義はFortranに似てます
    i = workitemIdx().x  # 今回は横方向にのみ値をアサイン
    c[i] = a[i] + b[i]
    return
end

@roc vadd!(c_d, a_d, b_d)
wait(@roc vadd!(c_d, a_d, b_d))

c_gpu = Array(c_d)  # 通常の配列に戻す
println("乱数値同士の和（GPU）: ", c_gpu, "\n")

println("err: ", c_gpu - c_cpu)

乱数値同士の和（CPU）: [0.574313705864145, 1.1132274652968963, 0.5343206021180691, 0.7460425325979447, 1.3808616734491492, 1.5667264094394697, 1.0802140784430854, 1.263793351120531, 1.0524503911857972, 0.39028883769414047, 1.1536641550351423, 0.6308321243304655, 0.636475104840817, 1.0340379088779634, 1.7584982662253594, 1.2710410272109878, 0.7540995301861237, 1.5991247149280603, 1.1480442782210754, 0.7527946226847653, 0.33329111262793765, 1.067224485716386, 1.03332932332275, 0.6323650275406112, 1.386644853447953, 1.297048872919008, 1.4163480212243362, 0.7487247932500827, 0.8802109697557805, 0.8006343757243919, 0.40926322488745104, 1.100590350367639, 0.547080379111766, 1.3621577121235304, 1.5354020485301532, 1.1250223441229883, 0.8490936078957303, 1.3785802067020037, 1.475735777086289, 1.5276189451342645, 0.0924798652315526, 1.496424980712552, 1.404065787224824, 0.7955414658149689, 1.0972752621947537, 1.110710319185356, 1.8707069635098197, 0.9796532072109876, 0.5322351810848407, 1.41106493025918